In [1]:
train_dir="/my/datasets/path/PACS/kfold"
fractal_dir="/my/datasets/path/deviantart"

# prompts=f"{prefix} art painting {suffix},{prefix} cartoon {suffix},{prefix} photo {suffix},{prefix} sketch {suffix}"
prompts = "art_painting,cartoon,photo,sketch"

! python ./main.py --train_dir {train_dir} --fractal_dir {fractal_dir} --prompts prompts --multi_domain true --num_images=1

^C


augmenting domain: art_painting



Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]c:\Users\Austi\anaconda3\envs\pytorch_gpu\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(

Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00, 15.38it/s]
c:\Users\Austi\anaconda3\envs\pytorch_gpu\lib\site-packages\PIL\TiffImagePlugin.py:793: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
Traceback (most recent call last):
  File "c:\Users\Austi\my_projects\GitHub\diffuseMix\main.py", line 94, in <module>
    main()
  File "c:\Users\Austi\my_projects\GitHub\diffuseMix\main.py", line 87, in main
    augment_domain(domain, domain_path, a